In [2]:
import cv2
import os
import pickle
import mediapipe as mp
from tqdm import tqdm
from IPython.display import clear_output

2025-03-26 13:44:11.126522: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742964251.218612    6919 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742964251.245301    6919 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742964251.452502    6919 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742964251.452520    6919 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742964251.452521    6919 computation_placer.cc:177] computation placer alr

In [ ]:
# 데이터셋 경로
DATASET_DIR = '/home/shin/deeplearning-repo-1/dataset/stand_knee_raise'

# 좌표 추출 운동 설정
exericse_name = "스탠딩 니업"

pickle_name = exericse_name + ".pkl"

exericse_type = ["가만히", "스탠딩 니업", "숄더프레스", "스쿼트", "프론트 런지", "푸시업", "브릿지", "스탠딩 바이시클", "사이드 런지"]
exericse_label = exericse_type.index(exericse_name)

# 시퀀스 사이즈
SEQUENCE_LENGTH = 20

# mediapipe 선언
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

sequences = []
labels = []
landmark_points = [ 11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28 ]

with mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5) as pose:
    for video_file in tqdm(os.listdir(DATASET_DIR)):
        video_path = os.path.join(DATASET_DIR, video_file)
        cap = cv2.VideoCapture(video_path)
        frames = []
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(frame_rgb)
            if results.pose_landmarks:
                landmarks = []

                lm = results.pose_landmarks.landmark

                # 기준점
                neck_x = (lm[11].x + lm[12].x) / 2
                neck_y = (lm[11].y + lm[12].y) / 2
                neck_z = (lm[11].z + lm[12].z) / 2

                for idx in landmark_points:
                    x = lm[idx].x - neck_x
                    y = lm[idx].y - neck_y
                    z = lm[idx].z - neck_z
                    landmarks.extend([x, y, z])
                frames.append(landmarks)
                if len(frames) >= SEQUENCE_LENGTH:
                    sequences.append(frames[-SEQUENCE_LENGTH:])
                    labels.append(exericse_label)
        cap.release()

I0000 00:00:1742964453.390304    6919 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1742964453.391735    7859 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
I0000 00:00:1742964453.412801    6919 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1742964453.416370    7874 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
  0%|          | 0/14 [00:00<?, ?it/s]

W0000 00:00:1742964453.484814    7848 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742964453.530235    7861 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742964453.538175    7845 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742964453.599064    7862 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
 21%|██▏       | 3/14 [01:08<04:54, 26.76s/it]

In [ ]:
with open(pickle_name, 'wb') as f:
    pickle.dump((sequences, labels), f)

    print(exericse_name + " 데이터 저장 완료!")

가만히 데이터 저장 완료!
